<a href="https://colab.research.google.com/github/Mansi-Nayak/Tools_Langchain/blob/main/Tools_Calling_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This script demonstrates how to build a simple LangChain-based tool-augmented assistant
using open-source models that run on CPU and are compatible with Google Colab.

Key Features:
1. Uses the HuggingFace `flan-t5-small` model for natural language generation.
2. Defines custom tools: `multiply` for arithmetic and `convert` for currency conversion.
3. Fetches real-time exchange rates via a public API (ExchangeRate-API).
4. Implements a `tool_dispatcher` function to detect tool-use intent in user queries.
5. Uses regular expressions to extract relevant numbers for processing.
6. Automatically chooses and invokes the right tool based on the input.
7. Provides natural language responses augmented with tool outputs.
8. Separates LLM reasoning from tool execution for reliability.
9. Built entirely with open-source libraries: `transformers`, `langchain`, and `requests`.
10. CPU-compatible and lightweight — ideal for running in Google Colab or local machines.
11. Handles basic math queries such as "Can you multiply 7 and 12?"
12. Supports currency queries like "Convert 100 INR to USD" by chaining API and math.
13. Demonstrates prompt + tool chaining in a modular, maintainable way.
14. Shows fallback messages when input cannot be interpreted correctly.
15. Allows easy extension with more tools in the future.
16. Avoids LangChain `bind_tools()` or `agent_executor` complexity.
17. Uses `.invoke()` for both LLM and tool calls.
18. All components work without requiring OpenAI or GPU access.
19. Promotes transparency by showing intermediate tool outputs.
20. Educational and customizable — good for beginners learning LangChain with HuggingFace.
"""

In [ ]:
!pip install -q langchain langchain-community transformers accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

In [ ]:
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableSequence
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline
import requests
import json
from typing import Annotated

In [14]:
# Load CPU-friendly model from HuggingFace
llm_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_new_tokens=128,
    device="cpu"
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [15]:
# Define a simple math tool
@tool
def multiply(a: int, b: int) -> int:
    """Returns the product of two numbers a and b"""
    return a * b

In [16]:
# Call the tool
print("Multiply Result:", multiply.invoke({'a': 3, 'b': 4}))

Multiply Result: 12


In [17]:
multiply.name

'multiply'

In [18]:
multiply.description

'Returns the product of two numbers a and b'

In [19]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [20]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    Fetch conversion rate from base to target currency using public API
    """
    url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
    res = requests.get(url)
    return res.json()

In [21]:
@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """Calculate converted value using a conversion rate"""
    return base_currency_value * conversion_rate

In [22]:
# Manual Tool Dispatcher
def tool_dispatcher(user_input: str):
    if "multiply" in user_input.lower():
        # Naive param extraction
        numbers = [int(s) for s in user_input.split() if s.isdigit()]
        if len(numbers) >= 2:
            return multiply.invoke({"a": numbers[0], "b": numbers[1]})
    elif "convert" in user_input.lower():
        if "inr" in user_input.lower() and "usd" in user_input.lower():
            rate_data = get_conversion_factor.invoke({'base_currency': 'INR', 'target_currency': 'USD'})
            rate = json.loads(rate_data.content).get('conversion_rate', 0)
            amount = [int(s) for s in user_input.split() if s.isdigit()]
            if amount:
                return convert.invoke({'base_currency_value': amount[0], 'conversion_rate': rate})
    return "No matching tool found."

In [23]:
# Run a few examples

# Example 1: Multiplication
query1 = "Can you multiply 7 and 12?"
print("User:", query1)
llm_response1 = llm.invoke(query1)
print("LLM:", llm_response1)
tool_output1 = tool_dispatcher(query1)
print("Tool Output:", tool_output1)

User: Can you multiply 7 and 12?
LLM: no
Tool Output: No matching tool found.


In [30]:
import re

def tool_dispatcher(user_input: str):
    input_lower = user_input.lower()

    # Match multiplication intent
    if "multiply" in input_lower:
        numbers = list(map(int, re.findall(r'\d+', user_input)))
        if len(numbers) >= 2:
            return multiply.invoke({"a": numbers[0], "b": numbers[1]})
        else:
            return "Found 'multiply' but couldn't extract two numbers."

    # Match currency conversion
    if "convert" in input_lower and "inr" in input_lower and "usd" in input_lower:
        amount = list(map(int, re.findall(r'\d+', user_input)))
        if amount:
            # Get exchange rate
            rate_data = get_conversion_factor.invoke({'base_currency': 'INR', 'target_currency': 'USD'})
            rate = rate_data.get('conversion_rate', 0)
            return convert.invoke({'base_currency_value': amount[0], 'conversion_rate': rate})
        else:
            return "Couldn't find amount to convert."

    return "No matching tool found."

In [31]:
query = "Can you multiply 7 and 12?"
print("User:", query)

llm_output = llm.invoke(query)
tool_result = tool_dispatcher(query)

# Construct final response
if isinstance(tool_result, int):
    response = f"Yes, {query.split('multiply')[1].strip()} is {tool_result}."
else:
    response = f"LLM said: {llm_output}\nTool Output: {tool_result}"

print("Final Response:", response)



User: Can you multiply 7 and 12?
Final Response: Yes, 7 and 12? is 84.


In [32]:
query = "Can you convert 100 INR to USD?"
print("User:", query)

llm_output = llm.invoke(query)
tool_result = tool_dispatcher(query)

# Construct final response
if isinstance(tool_result, float):
    amount = re.findall(r'\d+', query)
    response = f"Yes, {amount[0]} INR is approximately {round(tool_result, 2)} USD."
else:
    response = f"LLM said: {llm_output}\nTool Output: {tool_result}"

print("Final Response:", response)


User: Can you convert 100 INR to USD?
Final Response: Yes, 100 INR is approximately 1.15 USD.
